# Particle Set


Simulate trajectories of a particle cloud in a two-dimensional flow field.
A doubly-periodic domain and randomly-generated flow fields are initially used.
For additional documentation e.g. see :
[1](https://JuliaClimate.github.io/IndividualDisplacements.jl/dev/),
[2](https://JuliaClimate.github.io/MeshArrays.jl/dev/),
[3](https://docs.juliadiffeq.org/latest/solvers/ode_solve.html),
[4](https://en.wikipedia.org/wiki/Displacement_(vector))

Exercises:
- change the initial distribution of partices
- increase the duration of the trajectories simulation
- treat the non-periodic domain case by padding `u,v` with zeros
- replace `u,v` with your own two-dimensional flow fields

![particles in random flow](https://github.com/JuliaClimate/IndividualDisplacements.jl/raw/master/examples/figs/RandomFlow.gif)

## 1. Import Software

In [1]:
using IndividualDisplacements, DataFrames
p=dirname(pathof(IndividualDisplacements))
include(joinpath(p,"../examples/helper_functions.jl"))

Main.##256.isosurface

## 2. Flow Field

The `u,v` arrays below can be replaced with any other pair provided by the user.
A couple of important considerations, however:

- `u,v` are staggered on a C-grid; by `-0.5` grid point in direction `1` for `u` (`2` for `v`)
 from the grid cell center (0.5,0.5)
- `u,v` here derive from streamfunction `ϕ`, defined at the corner point, which ensures that
 the resulting `u,v` is non-divergent, purely rotational, over the C-grid domain.
In brief:

```
u=-(circshift(ϕ, (0,-1))-ϕ)
v=(circshift(ϕ, (-1,0))-ϕ)
```

In [2]:
u,v,ϕ=setup_random_flow()

([0.0058644817678497295 0.006440435122566035 … 0.0042205841020227786 0.005013693441245477; 0.0058422526446918616 0.0045552619328361535 … 0.005723492710805304 0.005554452468963012; … ; 0.007132145639489912 0.008272539999078336 … 0.003518441558138935 0.005722080604884459; 0.006826749828054153 0.007118755389116126 … 0.0031871815840456355 0.00484543729365651], [-0.011229160239335706 -0.011206931116177838 … -0.009185492602835646 -0.010688401211618171; -0.006548134051115612 -0.005884123279209268 … -0.0031975478904259502 -0.005219800117161909; … ; -0.012602154030271193 -0.012296758218835434 … -0.013810057315592442 -0.013478797341499142; -0.013937634299076337 -0.012975366238871914 … -0.012735975633510227 -0.01376937815148737], [0.033797828167713324 0.027933346399863595 … 0.04303210571098158 0.0388115216089588; 0.02256866792837762 0.016726415283685757 … 0.033846613108145934 0.02812312039734063; … ; 0.060337616497060854 0.05320547085757094 … 0.06957813866008425 0.06605969710194531; 0.04773546246

If user were to start with collocated velocity (`uC,vC` at the grid cell center) then
one can easily obtain the staggered velocity (`u,v`) as follows. These may contain both
[rotational and divergent](https://en.wikipedia.org/wiki/Helmholtz_decomposition) components.

```
u=0.5*(circshift(uC, (0,1))+uC)
v=0.5*(circshift(vC, (1,0))+vC)
```

## 3. Initialize Individuals

For example, we can initialize 100 particles within a central subdomain as follows.

In [3]:
np,nq=size(u)
x=np*(0. .+ 1.0*rand(1000))
y=nq*(0. .+ 1.0*rand(1000));

The `setup_point_cloud` function then wraps everything in the `Individuals` data structure.

In [4]:
𝐼=setup_point_cloud(u,v,X=x,Y=y)
𝐼.𝑃.𝑇[2]=10.
𝐼.🔴

,ID,x,y,t
,Int64,Float64,Float64,Float64


## 3. Compute Trajectories

The time period is `𝐼.𝑃.𝑇` by default, unless `∫!(𝐼,𝑇)` is called instead.

In [5]:
∫!(𝐼)

1×1000 Array{Array{Float64,1},2}:
 [8.19546, 9.55424, 1.0]  [5.24167, 16.1197, 1.0]  …  [3.90947, 9.97885, 1.0]

## 4. Plot Results

For example, generate a simple animation:

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*